![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FDev&file=aa+-+TFX+on+Vertex+AI+-+Interactive.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Dev/aa%20-%20TFX%20on%20Vertex%20AI%20-%20Interactive.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FDev%2Faa%2520-%2520TFX%2520on%2520Vertex%2520AI%2520-%2520Interactive.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Dev/aa%20-%20TFX%20on%20Vertex%20AI%20-%20Interactive.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Dev/aa%20-%20TFX%20on%20Vertex%20AI%20-%20Interactive.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# TFX on AI Platform - Interactive Pipeline

---
## Environment

In [1]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
import tfx
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.3.2
TFX version: 0.28.0


In [2]:
from absl import logging
#logging.set_verbosity(logging.INFO)  # Set default logging level.

---
## Setup

In [3]:
PIPELINE_NAME = 'my-pipeline'

# Make the Directory Structure for the Pipeline
LOCAL_DIR = PIPELINE_NAME
DATA_PATH = 'gs://statmike-mlops/digits/data'
MODULE_PATH = '{}/modules'.format(LOCAL_DIR)
PIPELINE_PATH = '{}/pipeline'.format(LOCAL_DIR)
METADATA_PATH = '{}/metadata/metadata.db'.format(LOCAL_DIR)
MODEL_PATH = '{}/model'.format(LOCAL_DIR)

In [4]:
!rm -rf {LOCAL_DIR}
!mkdir {LOCAL_DIR}
!mkdir {MODULE_PATH}

In [5]:
#from tfx.components import CsvExampleGen
from tfx.extensions.google_cloud_big_query.example_gen import component as big_query_example_gen_component
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.components import Pusher
from tfx.components import Trainer
from tfx.dsl.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto import example_gen_pb2

#from tfx.orchestration.local import local_dag_runner
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from typing import Optional, List, Text
import os

In [6]:
GOOGLE_CLOUD_REGION = 'us-central1'
GOOGLE_CLOUD_PROJECT='statmike-mlops'
GCS_BUCKET_NAME=GOOGLE_CLOUD_PROJECT
BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS = [
    '--project=' + GOOGLE_CLOUD_PROJECT,
    '--temp_location=' + os.path.join('gs://', GCS_BUCKET_NAME, 'tmp'),
    ]
BIG_QUERY_QUERY = """
      SELECT * FROM `statmike-mlops.digits.digits_prepped`
  """

---
## Pipeline

In [7]:
context=InteractiveContext(pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_PATH,
    beam_pipeline_args=BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS,
    metadata_connection_config=metadata.sqlite_metadata_connection_config(METADATA_PATH))

---
## Component: ExampleGen

In [8]:
query_train = """
  SELECT * FROM `statmike-mlops.digits.digits_prepped` WHERE SPLITS='TRAIN'
"""
query_test = """
  SELECT * FROM `statmike-mlops.digits.digits_prepped` WHERE SPLITS='TEST'
"""

input_pb2 = example_gen_pb2.Input(splits=[
  example_gen_pb2.Input.Split(name='train', pattern=query_train),
  example_gen_pb2.Input.Split(name='test', pattern=query_test)
])

example_gen = big_query_example_gen_component.BigQueryExampleGen(input_config=input_pb2)

In [9]:
context.run(example_gen)

/opt/conda/lib/python3.7/site-packages/apache_beam/io/gcp/bigquery.py:1881: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(


ExecutionResult(
    component_id: BigQueryExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "my-pipeline/pipeline/BigQueryExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "examples"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "BigQueryExampleGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [10]:
context.show(example_gen)

BigQueryExampleGen(spec: <tfx.types.standard_component_specs.QueryBasedExampleGenSpec object at 0x7fb317fb1b90>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x7fb317fb3250>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: BigQueryExampleGen, inputs: {}, outputs: {'examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 1
type_id: 5
uri: "my-pipeline/pipeline/BigQueryExampleGen/examples/1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"test\"]"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "examples"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "BigQueryExampleGen"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 5
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
)]
    additional_properties: {}
    additional_custom_properties: {}
)})

---
## Component: StatisticsGen

In [11]:
# Computes statistics over data for visualization and example validation.
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

In [12]:
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "my-pipeline/pipeline/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [13]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


---
## Component: SchemaGen

In [14]:
# Generates schema based on statistics files.
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],infer_feature_shape=True)

In [15]:
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "my-pipeline/pipeline/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [16]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'SPLITS',STRING,required,,'SPLITS'
'p0',FLOAT,required,,-
'p1',FLOAT,required,,-
'p10',FLOAT,required,,-
'p11',FLOAT,required,,-
...,...,...,...,...
'p7',FLOAT,required,,-
'p8',FLOAT,required,,-
'p9',FLOAT,required,,-


/home/jupyter/.local/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'SPLITS',"'TRAIN', 'TEST'"
'target_OE',"'Even', 'Odd'"


---
## Component: ExampleValidator

In [17]:
example_validator = ExampleValidator(statistics=statistics_gen.outputs['statistics'],
                                    schema=schema_gen.outputs['schema'])

In [18]:
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "my-pipeline/pipeline/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [19]:
context.show(example_validator.outputs['anomalies'])

---
## Component: Transform

In [20]:
schema_gen.outputs['schema'].get()[0].uri

'my-pipeline/pipeline/SchemaGen/schema/3'

In [21]:
schema_gen.outputs['schema'].get()[0].uri

'my-pipeline/pipeline/SchemaGen/schema/3'

In [22]:
%%writefile {MODULE_PATH}/transform.py
import tensorflow as tf
import tensorflow_transform as tft

TARGET = 'target'
OMIT = ['target_OE','SPLITS']

# TRANSFORM COMPONENT RUNS THIS FUNCTION
def preprocessing_fn(inputs):
    outputs = {}
    for x in inputs:
        if x == TARGET:
            # one hot encoding for the 10 digits values of the target
            outputs[TARGET] = tf.cast(tf.one_hot(tf.cast(inputs[TARGET],tf.int64),10),tf.float32)
        elif x not in OMIT:
            # each pixel normalized to mean=0 sd=1 scale
            outputs[x] = tft.scale_to_0_1(inputs[x])
    
    return outputs

Writing my-pipeline/modules/transform.py


In [23]:
transform = Transform(
                  examples=example_gen.outputs['examples'],
                  schema=schema_gen.outputs['schema'],
                  module_file=MODULE_PATH+'/transform.py')

In [24]:
context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 13
        uri: "my-pipeline/pipeline/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 5
        uri: "my-pipeline/pipeline/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"test\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "my-pipeline/pipeline/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [25]:
context.show(transform)

Transform(spec: <tfx.types.standard_component_specs.TransformSpec object at 0x7fb317c17310>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x7fb317aec750>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Transform, inputs: {'examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 1
type_id: 5
uri: "my-pipeline/pipeline/BigQueryExampleGen/examples/1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"test\"]"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "examples"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "BigQueryExampleGen"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 5
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
)]
    additional_properties: {}
    additional_custom_properties: {}
), 'schema': Channel(
    type_name: Schema
    artifacts: [Artifact(artifact: id: 3
type_id: 9
uri: "my-pipeline/pipeline/SchemaGen/schema/3"
custom_properties {
  key: "name"
  value {
    string_value: "schema"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "SchemaGen"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 9
name: "Schema"
)]
    additional_properties: {}
    additional_custom_properties: {}
)}, outputs: {'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: [Artifact(artifact: id: 5
type_id: 13
uri: "my-pipeline/pipeline/Transform/transform_graph/5"
custom_properties {
  key: "name"
  value {
    string_value: "transform_graph"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Transform"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 13
name: "TransformGraph"
)]
    additional_properties: {}
    additional_custom_properties: {}
), 'transformed_examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 6
type_id: 5
uri: "my-pipeline/pipeline/Transform/transformed_examples/5"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"test\"]"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "transformed_examples"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Transform"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 5
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
)]
    additional_properties: {}
    additional_custom_properties: {}
), 'updated_analyzer_cache': Channel(
    type_name: TransformCache
    artifacts: [Artifact(artifact: id: 7
type_id: 14
uri: "my-pipeline/pipeline/Transform/updated_analyzer_cache/5"
custom_properties {
  key: "name"
  value {
    string_value: "updated_analyzer_cache"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Transform"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 14
name: "TransformCache"
)]
    additional_properties: {}
    additional_custom_properties: {}
)})

In [26]:
transform.outputs['transform_graph'].get()[0].uri

'my-pipeline/pipeline/Transform/transform_graph/5'

---
## Component: Trainer

In [27]:
%%writefile {MODULE_PATH}/train.py


def get_model():
    
    feature_columns = []
    feature_layer_inputs = {}
    for header in selected_fields:
        if header != 'target':
            feature_columns.append(tf.feature_column.numeric_column(header))
            feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)
    
    feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()],outputs=tf.keras.layers.Dense(10,activation=tf.nn.softmax)(feature_layer_outputs))
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

def run_fn():
    # get data
    
    
    # get model
    model = get_model()
    
    # fit model
    model.fit(train,epochs=args.epochs)
    
    # save model
    model.save(args.model_dir,save_format='tf')
    

Writing my-pipeline/modules/train.py


In [ ]:
trainer = Trainer(
            module_file=MODULE_PATH+'/train.py',
            custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
            examples=transform.output['transformed_examples'],
            transform_graph=transform.outputs['transform_graph'],
            schema=schema_gen.outputs['schema'],
            train_args=,
            eval_args=
)

In [ ]:
context.run(trainer)

In [5]:
%%writefile {MODULE_PATH}/train.py
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf
from google.cloud import bigquery
import argparse
import os
import sys

parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--model-dir', dest='model_dir', default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--epochs',dest='epochs', default=10, type=int, help='Number of Epochs')
parser.add_argument('--batch_size',dest='batch_size', default=32, type=int, help='Batch Size')
#parser.add_argument('',dest='', default=, type=, help='')
args = parser.parse_args()

# built in parameters for data source:
#PROJECT_ID='statmike-mlops'
#BQDATASET_ID='digits'
#BQTABLE_ID='digits_prepped'

selected_fields = ['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'target']
output_types = ['FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'FLOAT64', 'INT64']

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != 'target':
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

from tensorflow.python.framework import dtypes
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

def transTable(row_dict):
    target=row_dict.pop('target')
    target = tf.one_hot(tf.cast(target,tf.int64),10)
    target = tf.cast(target,tf.float32)
    return(row_dict,target)

#client = BigQueryClient()
#session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TRAIN'",requested_streams=3)
#table = session.parallel_read_rows()
table = table.map(transTable)
train = table.shuffle(100000).batch(args.batch_size)

#client = BigQueryClient()
#session = client.read_session("projects/"+PROJECT_ID,PROJECT_ID,BQTABLE_ID,BQDATASET_ID,selected_fields,output_types,row_restriction="SPLITS='TEST'",requested_streams=3)
#table = session.parallel_read_rows()
table = table.map(transTable)
test = table.batch(args.batch_size)

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)
model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()],outputs=tf.keras.layers.Dense(10,activation=tf.nn.softmax)(feature_layer_outputs))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
#tf.keras.utils.plot_model(model,show_shapes=True, show_dtype=True)

history = model.fit(train,epochs=args.epochs)

model.save(args.model_dir)

Writing my-pipeline/modules/train.py


In [57]:
def mypipeline(pipeline_name: str, pipeline_path: str, data_path: str, query: str,
                     train_file: str, model_path: str,
                     metadata_path: str, beam_pipeline_args: Optional[List[Text]] = None) -> pipeline.Pipeline:

  # Brings data into the pipeline.
  #example_gen = CsvExampleGen(input_base=data_path)
  #example_gen = big_query_example_gen_component.BigQueryExampleGen(query=query)

  query_train = """
      SELECT * FROM `statmike-mlops.digits.digits_prepped` WHERE SPLITS='TRAIN'
  """
  query_test = """
      SELECT * FROM `statmike-mlops.digits.digits_prepped` WHERE SPLITS='TEST'
  """
    
  input_pb2 = example_gen_pb2.Input(splits=[
      example_gen_pb2.Input.Split(name='train', pattern=query_train),
      example_gen_pb2.Input.Split(name='test', pattern=query_test)
  ])
  example_gen = big_query_example_gen_component.BigQueryExampleGen(input_config=input_pb2)


  # Computes statistics over data for visualization and example validation.
  statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

  # Generates schema based on statistics files.
  schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'],infer_feature_shape=True)

  # Uses user-provided Python function that trains a model.
  trainer = Trainer(
      module_file=train_file,
      custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
      examples=example_gen.outputs['examples'],
      train_args=trainer_pb2.TrainArgs(num_steps=100),
      eval_args=trainer_pb2.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = Pusher(
      model=trainer.outputs['model'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=model_path)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      statistics_gen,
      schema_gen,
      #trainer,
      #pusher,
  ]

  return pipeline.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_path,
      metadata_connection_config=metadata.sqlite_metadata_connection_config(metadata_path),
      components=components,
      beam_pipeline_args=beam_pipeline_args)

In [ ]:
local_dag_runner.LocalDagRunner().run(mypipeline(
    pipeline_name=PIPELINE_NAME,
    pipeline_path=PIPELINE_PATH,
    data_path=DATA_PATH,
    query = BIG_QUERY_QUERY,
    train_file=MODULE_PATH+'/train.py',
    model_path=MODEL_PATH,
    beam_pipeline_args=BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS,
    metadata_path=METADATA_PATH
))